In [1]:
!python --version

Python 3.8.10


In [2]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
import optuna
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import joblib

import warnings
warnings.filterwarnings('ignore')

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
## modelling for silicon :
si_data = pd.read_csv('../data/cleaned/silicon.csv')
si_bandgap = pd.read_csv('../data/cleaned/bandgapSi.csv')

In [5]:
si_data.isnull().sum()

atomic_numbers_mean            0
atomic_numbers_max             0
atomic_numbers_min             0
atomic_numbers_std             0
volume_per_atom                0
                              ..
average_cationic_radius_std    0
average_anionic_radius_mean    0
average_anionic_radius_max     0
average_anionic_radius_min     0
average_anionic_radius_std     0
Length: 95, dtype: int64

In [6]:
si_data.shape

(5000, 95)

In [9]:
si_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 95 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   atomic_numbers_mean                           5000 non-null   float64
 1   atomic_numbers_max                            5000 non-null   int64  
 2   atomic_numbers_min                            5000 non-null   int64  
 3   atomic_numbers_std                            5000 non-null   float64
 4   volume_per_atom                               5000 non-null   float64
 5   empty_volume                                  5000 non-null   float64
 6   metals_fraction                               5000 non-null   float64
 7   distance_matrix_mean                          5000 non-null   float64
 8   distance_matrix_max                           5000 non-null   float64
 9   distance_matrix_min                           5000 non-null   f

In [7]:
si_data.head(2)

,atomic_numbers_mean,atomic_numbers_max,atomic_numbers_min,atomic_numbers_std,volume_per_atom,empty_volume,metals_fraction,distance_matrix_mean,distance_matrix_max,distance_matrix_min,distance_matrix_std,van_der_waals_radius_mean,van_der_waals_radius_max,van_der_waals_radius_min,van_der_waals_radius_std,electrical_resistivity_mean,electrical_resistivity_max,electrical_resistivity_min,electrical_resistivity_std,velocity_of_sound_mean,velocity_of_sound_max,velocity_of_sound_min,velocity_of_sound_std,reflectivity_mean,reflectivity_max,reflectivity_min,reflectivity_std,poissons_ratio_mean,poissons_ratio_max,poissons_ratio_min,poissons_ratio_std,molar_volume_mean,molar_volume_max,molar_volume_min,molar_volume_std,thermal_conductivity_mean,thermal_conductivity_max,thermal_conductivity_min,thermal_conductivity_std,melting_point_mean,melting_point_max,melting_point_min,melting_point_std,critical_temperature_mean,critical_temperature_max,critical_temperature_min,critical_temperature_std,superconduction_temperature_mean,superconduction_temperature_max,superconduction_temperature_min,superconduction_temperature_std,liquid_range_mean,liquid_range_max,liquid_range_min,liquid_range_std,bulk_modulus_mean,bulk_modulus_max,bulk_modulus_min,bulk_modulus_std,youngs_modulus_mean,youngs_modulus_max,youngs_modulus_min,youngs_modulus_std,brinell_hardness_mean,brinell_hardness_max,brinell_hardness_min,brinell_hardness_std,rigidity_modulus_mean,rigidity_modulus_max,rigidity_modulus_min,rigidity_modulus_std,vickers_hardness_mean,vickers_hardness_max,vickers_hardness_min,vickers_hardness_std,density_of_solid_mean,density_of_solid_max,density_of_solid_min,density_of_solid_std,coefficient_of_linear_thermal_expansion_mean,coefficient_of_linear_thermal_expansion_max,coefficient_of_linear_thermal_expansion_min,coefficient_of_linear_thermal_expansion_std,average_ionic_radius_mean,average_ionic_radius_max,average_ionic_radius_min,average_ionic_radius_std,average_cationic_radius_mean,average_cationic_radius_max,average_cationic_radius_min,average_cationic_radius_std,average_anionic_radius_mean,average_anionic_radius_max,average_anionic_radius_min,average_anionic_radius_std
0,13.000000,14,12,1.000000,18.856730,223.616458,0.083333,3.124381,4.791231,0.0,1.164230,1.73,1.73,1.73,0.0,4.400000e-08,4.400000e-08,4.400000e-08,0.0,4602.0,4602.0,4602.0,0.0,74.0,74.0,74.0,0.0,0.29,0.29,0.29,0.0,14.00,14.00,14.00,0.0,160.0,160.0,160.0,0.0,923.0,923.0,923.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,440.0,440.0,440.0,0.0,45.0,45.0,45.0,0.0,45.0,45.0,45.0,0.0,260.0,260.0,260.0,0.0,17.0,17.0,17.0,0.0,0.0,0.0,0.0,0.0,1738.0,1738.0,1738.0,0.0,0.000008,0.000008,0.000008,0.0,0.86,0.86,0.86,0.0,0.86,0.86,0.86,0.0,0.0,0.0,0.0,0.0
1,12.588235,41,8,9.052519,14.965699,1524.191713,0.009804,6.787983,12.008606,0.0,2.803602,2.18,2.18,2.18,0.0,1.520000e-07,1.520000e-07,1.520000e-07,0.0,3480.0,3480.0,3480.0,0.0,0.0,0.0,0.0,0.0,0.40,0.40,0.40,0.0,10.83,10.83,10.83,0.0,54.0,54.0,54.0,0.0,2750.0,2750.0,2750.0,0.0,0.0,0.0,0.0,0.0,9.25,9.25,9.25,0.0,2267.0,2267.0,2267.0,0.0,170.0,170.0,170.0,0.0,105.0,105.0,105.0,0.0,736.0,736.0,736.0,0.0,38.0,38.0,38.0,0.0,1320.0,1320.0,1320.0,0.0,8570.0,8570.0,8570.0,0.0,0.000007,0.000007,0.000007,0.0,0.82,0.82,0.82,0.0,0.82,0.82,0.82,0.0,0.0,0.0,0.0,0.0


In [8]:
X_train, X_test, y_train, y_test = train_test_split(si_data, si_bandgap, test_size=0.33, random_state=42)

In [11]:
X = X_train
y = y_train

In [15]:
X.reset_index(drop=True, inplace=True)
y.reset_index(drop=True, inplace=True)


In [18]:
X.head(2)

,atomic_numbers_mean,atomic_numbers_max,atomic_numbers_min,atomic_numbers_std,volume_per_atom,empty_volume,metals_fraction,distance_matrix_mean,distance_matrix_max,distance_matrix_min,distance_matrix_std,van_der_waals_radius_mean,van_der_waals_radius_max,van_der_waals_radius_min,van_der_waals_radius_std,electrical_resistivity_mean,electrical_resistivity_max,electrical_resistivity_min,electrical_resistivity_std,velocity_of_sound_mean,velocity_of_sound_max,velocity_of_sound_min,velocity_of_sound_std,reflectivity_mean,reflectivity_max,reflectivity_min,reflectivity_std,poissons_ratio_mean,poissons_ratio_max,poissons_ratio_min,poissons_ratio_std,molar_volume_mean,molar_volume_max,molar_volume_min,molar_volume_std,thermal_conductivity_mean,thermal_conductivity_max,thermal_conductivity_min,thermal_conductivity_std,melting_point_mean,melting_point_max,melting_point_min,melting_point_std,critical_temperature_mean,critical_temperature_max,critical_temperature_min,critical_temperature_std,superconduction_temperature_mean,superconduction_temperature_max,superconduction_temperature_min,superconduction_temperature_std,liquid_range_mean,liquid_range_max,liquid_range_min,liquid_range_std,bulk_modulus_mean,bulk_modulus_max,bulk_modulus_min,bulk_modulus_std,youngs_modulus_mean,youngs_modulus_max,youngs_modulus_min,youngs_modulus_std,brinell_hardness_mean,brinell_hardness_max,brinell_hardness_min,brinell_hardness_std,rigidity_modulus_mean,rigidity_modulus_max,rigidity_modulus_min,rigidity_modulus_std,vickers_hardness_mean,vickers_hardness_max,vickers_hardness_min,vickers_hardness_std,density_of_solid_mean,density_of_solid_max,density_of_solid_min,density_of_solid_std,coefficient_of_linear_thermal_expansion_mean,coefficient_of_linear_thermal_expansion_max,coefficient_of_linear_thermal_expansion_min,coefficient_of_linear_thermal_expansion_std,average_ionic_radius_mean,average_ionic_radius_max,average_ionic_radius_min,average_ionic_radius_std,average_cationic_radius_mean,average_cationic_radius_max,average_cationic_radius_min,average_cationic_radius_std,average_anionic_radius_mean,average_anionic_radius_max,average_anionic_radius_min,average_anionic_radius_std
0,9.6,20,6,3.912374,14.449911,860.788812,0.016667,4.559010,7.623971,0.0,1.371764,2.31,2.31,2.31,0.0,3.400000e-08,3.400000e-08,3.400000e-08,0.0,3810.0,3810.0,3810.0,0.0,0.0,0.0,0.0,0.0,0.31,0.31,0.31,0.0,26.20,26.20,26.20,0.0,200.0,200.0,200.0,0.0,1115.0,1115.0,1115.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,642.0,642.0,642.0,0.0,17.0,17.0,17.0,0.0,20.0,20.0,20.0,0.0,167.0,167.0,167.0,0.0,7.4,7.4,7.4,0.0,0.0,0.0,0.0,0.0,1550.0,1550.0,1550.0,0.0,0.000022,0.000022,0.000022,0.0,1.14,1.14,1.14,0.0,1.14,1.14,1.14,0.0,0.0,0.0,0.0,0.0
1,47.5,76,14,28.718461,163.267780,649.591979,0.250000,2.230172,4.473877,0.0,1.572374,2.06,2.06,2.06,0.0,1.270000e-07,1.270000e-07,1.270000e-07,0.0,5940.0,5940.0,5940.0,0.0,0.0,0.0,0.0,0.0,0.21,0.21,0.21,0.0,7.23,7.23,7.23,0.0,94.0,94.0,94.0,0.0,2180.0,2180.0,2180.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,764.0,764.0,764.0,0.0,160.0,160.0,160.0,0.0,279.0,279.0,279.0,0.0,1120.0,1120.0,1120.0,0.0,115.0,115.0,115.0,0.0,1060.0,1060.0,1060.0,0.0,7140.0,7140.0,7140.0,0.0,0.000005,0.000005,0.000005,0.0,0.94,0.94,0.94,0.0,0.94,0.94,0.94,0.0,0.0,0.0,0.0,0.0


In [20]:
X.shape

(3350, 95)

In [21]:
y.shape

(3350, 1)

In [19]:
y.head(2)

,band_gap
0,0.3747
1,0.0000


In [22]:
X = X.reset_index(drop=True)
y = y.reset_index(drop=True)


In [23]:
def objective(trial, X, y):
   
    param = {
                "n_estimators" : trial.suggest_int('n_estimators', 0, 500),
                'max_depth':trial.suggest_int('max_depth', 3, 5),
                'reg_alpha':trial.suggest_uniform('reg_alpha',0,6),
                'reg_lambda':trial.suggest_uniform('reg_lambda',0,2),
                'min_child_weight':trial.suggest_int('min_child_weight',0,5),
                'gamma':trial.suggest_uniform('gamma', 0, 4),
                'learning_rate':trial.suggest_loguniform('learning_rate',0.05,0.5),
                'colsample_bytree':trial.suggest_uniform('colsample_bytree',0.4,0.9),
                'subsample':trial.suggest_uniform('subsample',0.4,0.9)
            }
    
    cv = KFold(n_splits=5, shuffle=True, random_state=0)

    cv_scores = np.empty(5)
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        model = XGBRegressor(**param)
        model.fit(
            X_train,
            y_train,
            eval_set=[(X_test, y_test)],
            eval_metric="mae",
            early_stopping_rounds=100)
        preds = model.predict(X_test)
        cv_scores[idx] = mean_absolute_error(y_test, preds)

    return np.mean(cv_scores)

In [ ]:
study = optuna.create_study(direction="minimize", study_name="XGB regressor")
func = lambda trial: objective(trial, X, y)
study.optimize(func, n_trials=20, show_progress_bar=True)

[I 2024-08-19 12:22:41,283] A new study created in memory with name: XGB regressor


  0%|          | 0/20 [00:00<?, ?it/s]

[0]	validation_0-mae:1.42932
[1]	validation_0-mae:1.32657
[2]	validation_0-mae:1.24072
[3]	validation_0-mae:1.16785
[4]	validation_0-mae:1.11418
[5]	validation_0-mae:1.06636
[6]	validation_0-mae:1.02767
[7]	validation_0-mae:0.97943
[8]	validation_0-mae:0.94153
[9]	validation_0-mae:0.91329
[10]	validation_0-mae:0.88480
[11]	validation_0-mae:0.86663
[12]	validation_0-mae:0.84906
[13]	validation_0-mae:0.83208
[14]	validation_0-mae:0.82229
[15]	validation_0-mae:0.80757
[16]	validation_0-mae:0.79782
[17]	validation_0-mae:0.79116
[18]	validation_0-mae:0.77744
[19]	validation_0-mae:0.76943
[20]	validation_0-mae:0.76208
[21]	validation_0-mae:0.75573
[22]	validation_0-mae:0.75136
[23]	validation_0-mae:0.74509
[24]	validation_0-mae:0.74230
[25]	validation_0-mae:0.73902
[26]	validation_0-mae:0.73860
[27]	validation_0-mae:0.73540
[28]	validation_0-mae:0.73390
[29]	validation_0-mae:0.73026
[30]	validation_0-mae:0.72922
[31]	validation_0-mae:0.72799
[32]	validation_0-mae:0.72999
[33]	validation_0-ma